<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_2_Multihead_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Multhead Self-Attention**

This notebook builds a multihead self-attention mechanism as in figure 12.6

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [6]:
import numpy as np
import matplotlib.pyplot as plt

The multihead self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [7]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 6
# Number of dimensions of each input
D = 8
# Create an empty list
X = np.random.normal(size=(D,N))
# Print X
print(X)

[[ 1.78862847  0.43650985  0.09649747 -1.8634927  -0.2773882  -0.35475898]
 [-0.08274148 -0.62700068 -0.04381817 -0.47721803 -1.31386475  0.88462238]
 [ 0.88131804  1.70957306  0.05003364 -0.40467741 -0.54535995 -1.54647732]
 [ 0.98236743 -1.10106763 -1.18504653 -0.2056499   1.48614836  0.23671627]
 [-1.02378514 -0.7129932   0.62524497 -0.16051336 -0.76883635 -0.23003072]
 [ 0.74505627  1.97611078 -1.24412333 -0.62641691 -0.80376609 -2.41908317]
 [-0.92379202 -1.02387576  1.12397796 -0.13191423 -1.62328545  0.64667545]
 [-0.35627076 -1.74314104 -0.59664964 -0.58859438 -0.8738823   0.02971382]]


We'll need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4).  We'll use two heads, and (as in the figure), we'll make the queries keys and values of size D/H

In [8]:
# Number of heads
H = 2
# QDV dimension
H_D = int(D/H)

# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters for the first head
omega_q1 = np.random.normal(size=(H_D,D))
omega_k1 = np.random.normal(size=(H_D,D))
omega_v1 = np.random.normal(size=(H_D,D))
beta_q1 = np.random.normal(size=(H_D,1))
beta_k1 = np.random.normal(size=(H_D,1))
beta_v1 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters for the second head
omega_q2 = np.random.normal(size=(H_D,D))
omega_k2 = np.random.normal(size=(H_D,D))
omega_v2 = np.random.normal(size=(H_D,D))
beta_q2 = np.random.normal(size=(H_D,1))
beta_k2 = np.random.normal(size=(H_D,1))
beta_v2 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters
omega_c = np.random.normal(size=(D,D))

Now let's compute the multiscale self-attention

In [9]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [24]:
def concatenate_vertical(A, B):
    # Verifica se os arrays têm o mesmo número de colunas
    if len(A[0]) != len(B[0]):
        raise ValueError("Os arrays devem ter o mesmo número de colunas para concatenação vertical.")

    # Concatena os arrays verticalmente
    C = np.concatenate((A, B), axis=0)

    return C



In [32]:
 # Now let's compute self attention in matrix form
def multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c):

  # TODO Write the multihead scaled self-attention mechanism.
  # Replace this line
  X_prime = np.zeros_like(X) ;

  N = X.shape[1]
  vet1T = np.ones((N, 1)).T

  # 1. Compute queries, keys, and values
  V1 = np.matmul(beta_v1, vet1T) + np.matmul(omega_v1, X)
  Q1 = np.matmul(beta_q1, vet1T) + np.matmul(omega_q1, X)
  K1 = np.matmul(beta_k1, vet1T) + np.matmul(omega_k1, X)
    
  V2 = np.matmul(beta_v2, vet1T) + np.matmul(omega_v2, X)
  Q2 = np.matmul(beta_q2, vet1T) + np.matmul(omega_q2, X)
  K2 = np.matmul(beta_k2, vet1T) + np.matmul(omega_k2, X)

  # 2. Compute dot products
  dot_product1 = np.dot(K1.T,Q1)
  dot_product2 = np.dot(K2.T,Q2)
    
  # 3. Scale the dot products as in equation 12.9
  D_q = X.shape[0]/2 #the number of rows in omega_q and omega_k, which must be the same
  dot_product1 = dot_product1 / (np.sqrt(D_q))
  dot_product2 = dot_product2 / (np.sqrt(D_q))
    
  # 4. Apply softmax to calculate attentions
  smax1 = softmax_cols(dot_product1)
  smax2 = softmax_cols(dot_product2)
    
  # 5. Weight values by attentions
  X_prime1 = np.matmul(V1, smax1)
  X_prime2 = np.matmul(V2, smax2)  

  # 6. The outputs of these self-attention mechanisms are vertically concatenated, and another linear transform 
  # omega_c is applied to combine them
  X_conc = concatenate_vertical(X_prime1,X_prime2)
    
  X_prime = np.matmul(omega_c,X_conc)

  return X_prime

In [33]:
# Run the self attention mechanism
X_prime = multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c)

# Print out the results
np.set_printoptions(precision=3)
print("Your answer:")
print(X_prime)

print("True values:")
print("[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]")
print(" [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]")
print(" [  5.479   1.115   9.244   0.453   5.656   7.089]")
print(" [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]")
print(" [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]")
print(" [  3.548  10.036  -2.244   1.604  12.113  -2.557]")
print(" [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]")
print(" [  1.248  18.894  -6.409   3.224  19.717  -5.629]]")

# If your answers don't match, then make sure that you are doing the scaling, and make sure the scaling 
# value is correct

Your answer:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]
True values:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]
